In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import bokeh as bk
import tqdm
import pulp
import itertools

from pulp import GLPK
from IPython.core.display import display, HTML
%pylab inline

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(dfs, captions))
    for df, caption in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_excel("benxemydinh_distance.xlsx")
df

,Unnamed: 0,Company,Journeys,Times_per_day,Times_start,Times_end,Total_time,Payment,Start_point,Destination,Distances
0,0,XE KHÁCH BẢO YẾN,Hà Nội – Tuyên Quang,5,17:30,20:00,2 giờ 30 phút,100000,Hà Nội,Tuyên Quang,110.353031
1,1,XE KHÁCH BẢO YẾN,Hà Nội – Tuyên Quang,2,17:00,19:30,4 giờ,160000,Hà Nội,Tuyên Quang,110.353031
2,2,XE KHÁCH HẢI NAM,Hà Nội – Sơn La,6,16:00,18:30,8 giờ,180000,Hà Nội,Sơn La,202.624731
3,3,XE KHÁCH ĐẠI PHÁT,Hà Nội – Lào Cai,1,14:00,16:30,9 giờ,250000,Hà Nội,Lào Cai,245.815958
4,4,XE KHÁCH THANH LY,Hà Nội – Cao Bằng,3,12:05,14:35,9 giờ,200000,Hà Nội,Cao Bằng,188.108382
...,...,...,...,...,...,...,...,...,...,...,...
196,196,XE KHÁCH AN BÌNH TÂM,Hà Nội – Nghệ An,2,8:00,11:00,6 giờ 21 phút,200000,Hà Nội,Nghệ An,261.905830
197,197,XE KHÁCH ẤT HÀ,Hà Nội – Nghệ An,1,8:30,11:30,5 giờ 30 phút,170000,Hà Nội,Nghệ An,261.905830
198,198,XE KHÁCH ĐỨC LAN,Hà Nội – Nghệ An,7,9:00,12:00,5 giờ,170000,Hà Nội,Nghệ An,261.905830
199,199,XE KHÁCH HOÀNG ANH - NGHỆ AN,Hà Nội – Nghệ An,3,9:10,12:10,7 giờ,180000,Hà Nội,Nghệ An,261.905830


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     201 non-null    int64  
 1   Company        201 non-null    object 
 2   Journeys       201 non-null    object 
 3   Times_per_day  201 non-null    int64  
 4   Times_start    201 non-null    object 
 5   Times_end      201 non-null    object 
 6   Total_time     201 non-null    object 
 7   Payment        201 non-null    int64  
 8   Start_point    201 non-null    object 
 9   Destination    201 non-null    object 
 10  Distances      201 non-null    float64
dtypes: float64(1), int64(3), object(7)
memory usage: 17.4+ KB


In [4]:
#Define attributions of each verhicle
class Verhicle:
    def __init__(self, payment, distance, times_per_day, seats):
        self.payment = payment
        self.distance = distance
        self.times_per_day = times_per_day
        self.seats = seats

    def getPayment(self):
        return self.payment

    def getDistance(self):
        return self.distance

    def getTimesPerDay(self):
        return self.times_per_day

    def getSeats(self):
        return self.seats

    def __str__(self):
        return f"(payment={self.payment}, times={self.times_per_day}, distance={self.distance}, seats={self.seats})"

#Linear optimization using Pulp  
model = pulp.LpProblem(name="benxemydinh-opt",  sense=pulp.LpMaximize)

#Each verhicle have amount of passengers's weight and goods's weight which it's able to carry on
men_passenger = pulp.LpVariable(name="men_passenger", lowBound=0, cat="Integer")
women_passenger = pulp.LpVariable(name="women_passenger", lowBound=0, cat="Integer")
goods_weight = pulp.LpVariable(name="goods_weight", lowBound=0, cat="Float")

#Constant variable (in Viet Nam specified)
profit_of_goods = 2500 #in each kg
men_weight = 58 #medium
women_weight = 48 #medium
litter_per_km = 0.2 #medium

#Add column seats to data frame (df)
df = df.assign(Seats=pd.Series(np.random.randn(201)).values)

def getSeats(df):
    seats = []
    for i in range(201):
        for label in df.columns:
            if label == "Payment":
                if df[label][i] >= 200000:
                    seats.append(45)
                elif df[label][i] >= 120000:
                    seats.append(41)
                elif df[label][i] >= 80000:
                    seats.append(35)
                elif df[label][i] >= 40000:
                    seats.append(29)
    return seats

seats_per_verhicle = getSeats(df)
seats_per_verhicle[5]

def add_column_seats():
    for i in range(201):
        for j in range(len(seats_per_verhicle)):
            if i == j:
                df['Seats'][i] = seats_per_verhicle[j]
    return df

df = add_column_seats()

def verhicleDF(df):
    for i in range(201):
        for label in df.columns:
            if label == "Journeys":
                df[label][i] = Verhicle(
                    df['Payment'][i], df['Distances'][i], df['Times_per_day'][i], df['Seats'][i])
    return df

df = verhicleDF(df)
payments = [df['Journeys'][i].getPayment() for i in range(201)]
seats = [df['Journeys'][i].getSeats() for i in range(201)]
times = [df['Journeys'][i].getTimesPerDay() for i in range(201)]
distances = [df['Journeys'][i].getDistance() for i in range(201)]
# print(payments)


journeys_df = df[["Company", "Payment", "Seats", "Times_per_day", "Distances"]].copy()
journeys_df.set_index(["Company"], inplace = True)
print(journeys_df)

# attributions_journey = {'Payment': payments,
#                         'Seats': seats,
#                         'Times': times,
#                         'Distances': distances}
# companys_list = journeys_df.index.to_list()
# attributions_list = journeys_df.columns.to_list()
# var_dict = pulp.LpVariavles.dict(name = "Optimization",
#                                 indexs = [(c,a) for c in companys_list for a in attributions_list],
#                                 cat = "Binary")
# #Maximize for each company
# model = pulp.lpSum([journeys_df.loc[c,a] * var_dict[(c,a)] for c in companys_list for a in attributions_list])

# solution = []

# for num in tqdm(range(201)):
    
    
#Opt function
model += pulp.lpSum([payments[1] * (men_passenger + women_passenger) * times[1]
                    + profit_of_goods * goods_weight
                    + litter_per_km * distances[1] * times[1] * 2])
#Subjects:
model += pulp.lpSum([men_passenger + women_passenger]) <=  [seats[1]]
model += pulp.lpSum([men_weight * men_passenger + women_weight * women_passenger + goods_weight]) <= 2000
model += pulp.lpSum([men_passenger]) >= seats[1]/3
model.solve()

print("Status: ",  pulp.LpStatus[model.status])

for v in model.variables():
    print(v.name, "=", v.varValue)

print("Total Benefit of", df["Company"][1],"in one day =", pulp.value(model.objective))
df.head()

                              Payment  Seats  Times_per_day   Distances
Company                                                                
XE KHÁCH BẢO YẾN               100000   35.0              5  110.353031
XE KHÁCH BẢO YẾN               160000   41.0              2  110.353031
XE KHÁCH HẢI NAM               180000   41.0              6  202.624731
XE KHÁCH ĐẠI PHÁT              250000   45.0              1  245.815958
XE KHÁCH THANH LY              200000   45.0              3  188.108382
...                               ...    ...            ...         ...
XE KHÁCH AN BÌNH TÂM           200000   45.0              2  261.905830
XE KHÁCH ẤT HÀ                 170000   41.0              1  261.905830
XE KHÁCH ĐỨC LAN               170000   41.0              7  261.905830
XE KHÁCH HOÀNG ANH - NGHỆ AN   180000   41.0              3  261.905830
XE KHÁCH PHAN HIỀN             150000   41.0              1  261.905830

[201 rows x 4 columns]
Status:  Optimal
goods_weight = 36.0
men

,Unnamed: 0,Company,Journeys,Times_per_day,Times_start,Times_end,Total_time,Payment,Start_point,Destination,Distances,Seats
0,0,XE KHÁCH BẢO YẾN,"(payment=100000, times=5, distance=110.3530305...",5,17:30,20:00,2 giờ 30 phút,100000,Hà Nội,Tuyên Quang,110.353031,35.0
1,1,XE KHÁCH BẢO YẾN,"(payment=160000, times=2, distance=110.3530305...",2,17:00,19:30,4 giờ,160000,Hà Nội,Tuyên Quang,110.353031,41.0
2,2,XE KHÁCH HẢI NAM,"(payment=180000, times=6, distance=202.6247313...",6,16:00,18:30,8 giờ,180000,Hà Nội,Sơn La,202.624731,41.0
3,3,XE KHÁCH ĐẠI PHÁT,"(payment=250000, times=1, distance=245.8159581...",1,14:00,16:30,9 giờ,250000,Hà Nội,Lào Cai,245.815958,45.0
4,4,XE KHÁCH THANH LY,"(payment=200000, times=3, distance=188.1083816...",3,12:05,14:35,9 giờ,200000,Hà Nội,Cao Bằng,188.108382,45.0
